In [62]:
# Definition Block
furl = 'http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt={:02d}-{}-{}&todt={:02d}-{}-{}'
forced = False
dataFilenameFormat = 'amfiData/{}{:02d}{:02d}-{}{:02d}{:02d}.txt'
amfiDataPath = 'amfiData/{}'
amfiDataDir = './amfiData/'
amfiDataLookFor = 'Scheme Code;Scheme Name;ISIN Div Payout/ISIN Growth;ISIN Div Reinvestment;Net Asset Value;Repurchase Price;Sale Price;Date'
sleepDurationForAPI = 3
earliestRecordedYear = 2005
jsonFilenameFormat = 'amfiJSON/{}.json'
amfiJSONDir = './amfiJSON/'
amfiJSONPath = 'amfiJSON/{}'
months = dict([[x[1], x[0]] for x in [
    ['Jan', 1],
    ['Feb', 2],
    ['Mar', 3],
    ['Apr', 4],
    ['May', 5],
    ['Jun', 6],
    ['Jul', 7],
    ['Aug', 8],
    ['Sep', 9],
    ['Oct', 10],
    ['Nov', 11],
    ['Dec', 12]
]])

In [58]:
import requests
import time
import os
from datetime import date

if not (furl and dataFilenameFormat and sleepDurationForAPI and amfiDataDir):
    raise ValueError(" Run # Definition BLock first")


filesPresent = [amfiDataPath.format(f) for f in os.listdir(amfiDataDir) if f.endswith('.txt')]
# print(filesPresent)

daysList = [(2, 8), (9, 15), (16, 23), (24, 27), (28, 1)]

today = date.today()
currentYear = today.year
currentMonth = today.month
currentDay = today.day

print(currentYear, currentMonth, currentDay)
onceMonth = True
onceDay = currentDay < 28

for yyyy in range(currentYear, earliestRecordedYear, -1):
    monthsToSeek = months
    if onceMonth:
        onceMonth = False
        if currentDay == 1:
            onceDay = False
            if currentMonth > 1:
                monthsToSeek = {
                    k: v for k, v in months.items() if k <= currentMonth - 1
                }
            else:
                continue
        else:
            monthsToSeek = {
                k: v for k, v in months.items() if k <= currentMonth
            }
    for index, month in monthsToSeek.items():
        for days in daysList:
            if onceDay:
                if currentDay < days[0]:
                    onceDay = False
                    continue
            yyyy1 = yyyy
            yyyy2 = yyyy
            am = index
            if days[0] < days[1]:
                url = furl.format(days[0], month, yyyy1, days[1], month, yyyy2)
            else:
                if month == 'Dec':
                    am = 1
                    monthx = 'Jan'
                    yyyy2 += 1
                else:
                    am += 1
                    monthx = months[index]
                url = furl.format(days[0], month, yyyy1, days[1], monthx, yyyy2)
            filename = dataFilenameFormat.format(yyyy1, index, days[0], yyyy2, am, days[1])
            if filename in filesPresent and not forced:
                print("Skipping: {}, already present".format(filename))
                continue
            print(url)
            file = requests.get(url, allow_redirects=True)
            print(filename)
            open(filename, 'wb').write(file.content)
            time.sleep(sleepDurationForAPI)


2021 7 31
Skipping: amfiData/20210102-20210108.txt, already present
Skipping: amfiData/20210109-20210115.txt, already present
Skipping: amfiData/20210116-20210123.txt, already present
Skipping: amfiData/20210124-20210127.txt, already present
Skipping: amfiData/20210128-20210201.txt, already present
Skipping: amfiData/20210202-20210208.txt, already present
Skipping: amfiData/20210209-20210215.txt, already present
Skipping: amfiData/20210216-20210223.txt, already present
Skipping: amfiData/20210224-20210227.txt, already present
Skipping: amfiData/20210228-20210301.txt, already present
Skipping: amfiData/20210302-20210308.txt, already present
Skipping: amfiData/20210309-20210315.txt, already present
Skipping: amfiData/20210316-20210323.txt, already present
Skipping: amfiData/20210324-20210327.txt, already present
Skipping: amfiData/20210328-20210401.txt, already present
Skipping: amfiData/20210402-20210408.txt, already present
Skipping: amfiData/20210409-20210415.txt, already present
Skip

amfiData/20060102-20060108.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=09-Jan-2006&todt=15-Jan-2006
amfiData/20060109-20060115.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=16-Jan-2006&todt=23-Jan-2006
amfiData/20060116-20060123.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=24-Jan-2006&todt=27-Jan-2006
amfiData/20060124-20060127.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=28-Jan-2006&todt=01-Jan-2006
amfiData/20060128-20060201.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=02-Feb-2006&todt=08-Feb-2006
amfiData/20060202-20060208.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=09-Feb-2006&todt=15-Feb-2006
amfiData/20060209-20060215.txt
http://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt=16-Feb-2006&todt=23-Feb-2006
amfiData/20060216-20060223.txt
http://portal.amfiindia.com/DownloadNAVHistor

In [59]:
import os

if not (furl and dataFilenameFormat and sleepDurationForAPI and amfiDataDir):
    raise ValueError(" Run # Definition BLock first")

filesPresent = [amfiDataPath.format(f) for f in os.listdir(amfiDataDir) if f.endswith('.txt')]
lf = []
scontent = set()
for f in filesPresent:
    with open(f) as of:
        contents = of.read().split('\n')
        if contents[0] != amfiDataLookFor:
            lf.append(f)
        i = 0
        while contents[i] == '':
            i += 1
        scontent.add(contents[i])
lf.sort()
for s in scontent:
    print(s)
print(len(lf))
for f in lf:
    print(f)
#     os.remove(f)

Scheme Code;Scheme Name;ISIN Div Payout/ISIN Growth;ISIN Div Reinvestment;Net Asset Value;Repurchase Price;Sale Price;Date
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Frameset//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-frameset.dtd">
21
amfiData/20060102-20060108.txt
amfiData/20060109-20060115.txt
amfiData/20060116-20060123.txt
amfiData/20060124-20060127.txt
amfiData/20060128-20060201.txt
amfiData/20060202-20060208.txt
amfiData/20060209-20060215.txt
amfiData/20060216-20060223.txt
amfiData/20060224-20060227.txt
amfiData/20060228-20060301.txt
amfiData/20060302-20060308.txt
amfiData/20060309-20060315.txt
amfiData/20060316-20060323.txt
amfiData/20060324-20060327.txt
amfiData/20210128-20210201.txt
amfiData/20210228-20210301.txt
amfiData/20210328-20210401.txt
amfiData/20210428-20210501.txt
amfiData/20210528-20210601.txt
amfiData/20210628-20210701.txt
amfiData/20210728-20210801.txt


In [60]:
import os

if not (furl and dataFilenameFormat and sleepDurationForAPI and amfiDataDir):
    raise ValueError(" Run # Definition BLock first")

filesPresent = [amfiStorePath.format(f) for f in os.listdir(amfiDataDir) if f.endswith('.txt')]
lf = []
scontent = set()
for f in filesPresent:
    with open(f) as of:
        contents = of.read().split('\n')
        if contents[0] != amfiDataLookFor:
            lf.append(f)
        i = 0
        while contents[i] == '':
            i += 1
        scontent.add(contents[i])
lf.sort()
for s in scontent:
    print(s)
print(len(lf))
for f in lf:
    print(f)
    os.remove(f)

Scheme Code;Scheme Name;ISIN Div Payout/ISIN Growth;ISIN Div Reinvestment;Net Asset Value;Repurchase Price;Sale Price;Date
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Frameset//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-frameset.dtd">
21
amfiData/20060102-20060108.txt
amfiData/20060109-20060115.txt
amfiData/20060116-20060123.txt
amfiData/20060124-20060127.txt
amfiData/20060128-20060201.txt
amfiData/20060202-20060208.txt
amfiData/20060209-20060215.txt
amfiData/20060216-20060223.txt
amfiData/20060224-20060227.txt
amfiData/20060228-20060301.txt
amfiData/20060302-20060308.txt
amfiData/20060309-20060315.txt
amfiData/20060316-20060323.txt
amfiData/20060324-20060327.txt
amfiData/20210128-20210201.txt
amfiData/20210228-20210301.txt
amfiData/20210328-20210401.txt
amfiData/20210428-20210501.txt
amfiData/20210528-20210601.txt
amfiData/20210628-20210701.txt
amfiData/20210728-20210801.txt


In [78]:
import os
import json

if not (furl and dataFilenameFormat and sleepDurationForAPI and amfiDataDir):
    raise ValueError(" Run # Definition BLock first")

filesPresent = [amfiDataPath.format(f) for f in os.listdir(amfiDataDir) if f.endswith('.txt')]
countHEADMap = {}
categories = set()
mfhs = set()
mf = {}
mfd = {}
currentMFH = ''
currentCategory = ''
for f in filesPresent:
    with open(f) as of:
        contents = of.read().split('\n')
        count = 0
        lastLine = ''
        for c in contents:
            if c == '':
                continue
            if amfiDataLookFor in c:
                count += 1
            elif ';' in c:
                if lastLine != '':
                    mfhs.add(lastLine)
                    currentMFH = lastLine
                    lastLine = ''
#                 else:
#                     print(f, c)
                data = c.split(';')
                if len(data) != 8:
                    raise ValueError()
                schemeCode = int(data[0])
                if schemeCode not in mf:
                    mf[schemeCode] = {
                        'schemeCode': schemeCode,
                        'schemeName': data[1],
                        'ISIN_Div_Payout_or_ISIN_Growth': data[2] if len(data[2]) else 'NA',
                        'ISIN_Div_Reinvestment': data[3] if len(data[3]) else 'NA',
                        'amc': currentMFH,
                        'category': currentCategory
                    }
                else:
                    td = {
                        'schemeCode': schemeCode,
                        'schemeName': data[1],
                        'ISIN_Div_Payout_or_ISIN_Growth': data[2] if len(data[2]) else 'NA',
                        'ISIN_Div_Reinvestment': data[3] if len(data[3]) else 'NA',
                        'amc': currentMFH,
                        'category': currentCategory
                    }
                    for k in td:
                        if td[k] != mf[schemeCode][k]:
                            print(f, k, k)
                nav = 'NA'
                try:
                    nav = float(data[4])
                except:
                    nav = 'NA'
                repurchasePrice = 'NA'
                try:
                    repurchasePrice = float(data[5])
                except:
                    repurchasePrice = 'NA'
                salePrice = 'NA'
                try:
                    salePrice = float(data[6])
                except:
                    salePrice = 'NA'
                dataDate = data[7]
                dateSplit = dataDate.split('-')
                if len(dateSplit) != 3:
                    raise ValueError("{} date failed:{}".format(f, c))
                d = {
                    'nav': nav,
                    'repurchasePrice': repurchasePrice,
                    'salePrice': salePrice,
                    'date': dataDate
                }
                if nav == 'NA' and repurchasePrice == 'NA' and 'salePrice' == 'NA':
                    continue
                if schemeCode not in mfd:
                    mfd[schemeCode] = []
                mfd[schemeCode].append(d)
            else:
                if lastLine != '':
                    currentCategory = lastLine
                    categories.add(lastLine)
                lastLine = c
#         print(f, count)
        if count not in countHEADMap:
            countHEADMap[count] = 0
        countMap[count] += 1
print("countMap: ", countMap)
print("categories: ", len(categories), '' in categories)
print("mfhs: ", len(mfhs), '' in mfhs)
print("mf: ", len(mf))
print("mfd: ", len(mfd))
for sc in mf:
    if sc not in mfd:
        raise ValueError("{} present in mf but in mfd".format(sc))
for sc in mfd:
    if sc not in mf:
        raise ValueError("{} present in mf but in mfd".format(sc))

def dateNumber(d):
    d = d.split('-')
    d[1] = [x[0] for x in months.items() if x[1] == d[1]]
    assert len(d[1]) == 1, f'{d[1]}'
    d[1] = d[1][0]
    n = '{}{:02d}{}'.format(*d[::-1])
    return int(n)

mfl = []
for sc in mf:
    d = {
        'meta': mf[sc],
        'data': sorted(mfd[sc], key=lambda entry: dateNumber(entry['date']), reverse=True)
    }
    mfl.append(mf[sc])
    with open(jsonFilenameFormat.format(sc), 'w') as jsf:
#         json.dump(d, jsf, indent = 4, sort_keys=True)
        json.dump(d, jsf)
    
with open(jsonFilenameFormat.format('ALLMF'), 'w') as jsf:
#         json.dump(d, jsf, indent = 4, sort_keys=True)
    json.dump(mfl, jsf)

countMap:  {1: 4707}
categories:  54 False
mfhs:  56 False
mf:  10649
mfd:  10649


In [91]:
import requests
import json

url = 'https://api.mfapi.in/mf'
# file = requests.get(url, allow_redirects=True)
filename = jsonFilenameFormat.format('ALLMFi')
print(filename)
# open(filename, 'wb').write(file.content)
open(filename, 'r').read()[:20]
kid = json.load(open(filename, 'r'))
filename = jsonFilenameFormat.format('ALLMF')
open(filename, 'r').read()[:20]
kd = json.load(open(filename, 'r'))
ki = set([x['schemeCode'] for x in kid])
k = set([x['schemeCode'] for x in kd])
print(len(k), len(ki), len(kd), len(kid), k - ki, list(ki - k)[:4])

amfiJSON/ALLMFi.json
10649 33370 10649 41850 {120633, 120626, 130953, 109943} [131072, 131073, 131074, 131076]


In [104]:
import requests
import json
  
# Making a POST request

filename = jsonFilenameFormat.format('ALLMFi')
kd = json.load(open(filename, 'r'))
print(len(json.dumps(kd)))
r = requests.post('https://api.jsonstorage.net/v1/json', data={'data':kd})
  
# check status code for response recieved
# success code - 200
print(r)
  
# print content of request
print(r.json())

4478122
<Response [400]>
{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.1', 'title': 'One or more validation errors occurred.', 'status': 400, 'traceId': '00-8d50caecebf7fd4bb328fc35253048db-c0ac435f0c188349-00', 'errors': {'': ['Failed to read the request form. Form value count limit 1024 exceeded.']}}
